In [1]:
!pip install altair

In [0]:
import altair as alt
import pandas as pd
import json

In [0]:
def loadData():
    """Load data from events_modified timestamp
    
  
    """
    import os
    cur_dir = os.path.dirname(__file__)
    ios_reviews = csv.load(open(os.path.join(cur_dir, 'mod_ts_vehicle_paths_pnas.csv'), 'r'))


    ## We ignore the 
    dfIOS = pd.DataFrame(((app, review['rating'], review['review'])
                          for app,reviews in ios_reviews.items() 
                          for review in reviews), columns=['name', 'rating', 'content'])
    
    return {'ios': dfIOS}


In [0]:
import os
import csv
cur_dir = os.path.dirname(os.getcwd())
df = pd.read_csv('mod_ts_vehicle_paths_pnas.csv')

In [15]:
df.head()

,Unnamed: 0,Vehicle_ID,Latitude,Longitude,Num_Passengers,Timestamp_dt
0,0,137,40.73467,-73.99459,0,2013-10-05 05:00:30
1,1,321,40.78573,-73.97644,0,2013-10-05 05:00:30
2,2,428,40.78154,-73.96041,0,2013-10-05 05:00:30
3,3,148,40.73478,-73.99871,0,2013-10-05 05:00:31
4,4,148,40.73478,-73.99871,0,2013-10-05 05:00:31


In [0]:
df['ts'] = pd.to_datetime(df['Timestamp_dt'])

In [0]:
df_ge4 = df[df['Num_Passengers'] > 4]

In [19]:
df_ge4['hour'] = df['ts'].dt.hour

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_ge4['week'] = df['ts'].dt.day

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
df_ge4.head()

,Unnamed: 0,Vehicle_ID,Latitude,Longitude,Num_Passengers,Timestamp_dt,ts,hour,week
13905,13905,251,40.74897,-74.00324,5,2013-10-05 05:11:05,2013-10-05 05:11:05,5,5
13906,13906,251,40.74959,-74.00279,5,2013-10-05 05:11:10,2013-10-05 05:11:10,5,5
13907,13907,251,40.75022,-74.00232,5,2013-10-05 05:11:15,2013-10-05 05:11:15,5,5
16114,16114,432,40.74400,-73.98570,5,2013-10-05 05:10:20,2013-10-05 05:10:20,5,5
16115,16115,432,40.74462,-73.98525,5,2013-10-05 05:10:30,2013-10-05 05:10:30,5,5


In [0]:
hgby = df_ge4[['hour']].groupby(df_ge4['hour']).count()

In [0]:
hgby.rename(columns={'hour': 'hour_count'},inplace=True)

In [0]:
hgby.reset_index(inplace=True)

In [0]:
hbgy_num_pass = df_ge4[['Num_Passengers','hour','ts']].groupby(['Num_Passengers','hour']).count()

In [0]:
hbgy_num_pass.reset_index(inplace=True)

In [0]:
hbgy_num_pass.to_csv('pass_hourly.csv')

In [0]:
pass_hourly = hbgy_num_pass.pivot(index='Num_Passengers',columns='hour',values='ts')

In [0]:
pass_hourly.fillna(0,inplace=True)

In [65]:
pass_hourly.head()

hour,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Num_Passengers,,,,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,9954.0,19554.0,10496.0,4772.0,2894.0,3505.0,5897.0,4793.0,943.0,141.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,4565.0,15463.0,10433.0,6164.0,3635.0,3043.0,6883.0,5959.0,1884.0,320.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,2171.0,11601.0,10404.0,6299.0,4026.0,3477.0,7180.0,6900.0,3034.0,604.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,881.0,7221.0,9145.0,6614.0,4090.0,4431.0,7127.0,8310.0,4074.0,1141.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,380.0,4473.0,7279.0,5517.0,3781.0,4560.0,6484.0,9853.0,5657.0,1945.0,469.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
color_expression    = "highlight._vgsid_==datum._vgsid_"
color_condition     = alt.ConditionalPredicateValueDef(color_expression, "SteelBlue")

In [0]:
highlight_selection = alt.selection_single(name="highlight", empty="all", on="mouseover")

In [0]:
maxCount            = int(hgby['hour'].value_counts().max())

In [83]:
hgby.head()

,hour,hour_count
0,0,121525
1,1,123027
2,2,124549
3,3,125307
4,4,123005


In [0]:
hgby.to_csv('hourly_overpass.csv')

In [0]:
pass_hourly.to_csv('pass_hourly.csv')

In [0]:
from google.colab import files

In [0]:
files.download('pass_hourly.csv')

In [0]:
files.download('hourly_overpass.csv')

In [0]:
d

In [0]:
def showRatingDistribution(data, name=''):
    """Create an interactive visualization showing the distribution of ratings
    
    Args:
        data (DataFrame): the input data frame that must at least consists
            two columns 'name' and 'rating' for app names and ratings.
        name (str): the name of the platform (optional) to be displayed.

    Return:
        Chart: an Altair chart object that corresponds to the visualization
    """
    ## The color expression for highlighting the bar under mouse
    color_expression    = "highlight._vgsid_==datum._vgsid_"
    color_condition     = alt.ConditionalPredicateValueDef(color_expression, "SteelBlue")
    
    ## There are two types of selection in our chart:
    ## (1) A selection for highlighting a bar when the mouse is hovering over
    highlight_selection = alt.selection_single(name="highlight", empty="all", on="mouseover")
    
    ## (2) A selection for updating the rating distribution when the mouse is clicked
    ## Note the encodings=['y'] parameter is needed to specify that once a selection
    ## is triggered, it will propagate the encoding channel 'y' as a condition for
    ## any subsequent filter done on this selection. In short, it means use the data
    ## field associated with the 'y' axis as a potential filter condition.
    rating_selection    = alt.selection_single(name="hour_count", empty="all", encodings=['y'])
    
    ## We need to compute the max count to scale our distribution appropriately
    maxCount            = int(data['hour_count'].value_counts().max())

    ## Our visualization consists of two bar charts placed side by side. The first one
    ## sorts the apps by their average ratings as below. Note the compound selection
    ## that is constructed by adding the two selections together.
    barMean = alt.Chart() \
        .mark_bar(stroke="Black") \
        .encode(
            alt.X("mean(rating):Q", axis=alt.Axis(title="Average Rating", tickCount=min(maxCount, 6))),
            alt.Y('name:O', axis=alt.Axis(title="{} App Name".format(name)), 
                  sort=alt.EncodingSortField(field="rating", op="mean", order='descending')),
            alt.ColorValue("LightGrey", condition=color_condition),
        ) \
        .add_selection(highlight_selection) \
        .add_selection(rating_selection)

    
        
    ## We just need to concatenate the plots horizontally, and return the result.
    return alt.hconcat(barMean, barRating,
        data=data,
        title="{} App Ratings".format(name)
    )